In [1]:
import os

In [2]:
%pwd

'd:\\Capestone projects\\Project_2\\document_summarization_for_legal_texts\\research'

In [3]:
os.chdir("..//")

In [4]:
%pwd

'd:\\Capestone projects\\Project_2\\document_summarization_for_legal_texts'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion Configuration
    """
    root_dir: Path
    source_URL: str
    locate_data_file: Path
    unzip_dir: Path

In [6]:
from datascience.constants import *
from datascience.utils.common import read_yaml, create_directories

In [7]:
class configuration_manager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Returns Data Ingestion Configuration
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir, config.unzip_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            locate_data_file=Path(config.locate_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from datascience.logging import logger
from datascience.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.locate_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.locate_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.locate_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.locate_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
import urllib.request
from pathlib import Path
import zipfile

url = 'https://github.com/madhusudhanant/Datasets/raw/main/legal_documents.zip'
output_path = Path('artifacts/data_ingestion/Legal_texts.zip')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Download the file
print(f'Downloading from {url}...')
urllib.request.urlretrieve(url, output_path)
print(f'Downloaded to {output_path}')

# Check if the file is a valid zip
try:
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        print('ZIP file is valid!')
except Exception as e:
    print(f'ZIP file is invalid: {e}')

Downloaded to artifacts\data_ingestion\Legal_texts.zip
ZIP file is valid!


In [13]:
try:
    config = configuration_manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-01 21:30:35,540] -> [INFO]: common - yaml file: config\config.yaml loaded successfully
[2025-07-01 21:30:35,546] -> [INFO]: common - yaml file: params.yaml loaded successfully
[2025-07-01 21:30:35,550] -> [INFO]: common - created directory at: artifacts
[2025-07-01 21:30:35,552] -> [INFO]: common - created directory at: artifacts/data_ingestion
[2025-07-01 21:30:35,555] -> [INFO]: common - created directory at: artifacts/data_ingestion
[2025-07-01 21:30:35,558] -> [INFO]: 1617986658 - File already exists of size: ~ 790 KB
